# 1) Data Prep

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)

0 C:\Users\p.schambach\Desktop\DSR\drone_steering\models
1 C:\Users\p.schambach\Desktop\DSR\drone_steering


## a) Data loading

In [3]:
from app_local.module import DataEnsembler, GestureTransformer

de = DataEnsembler(120)
de.investigate_available_datafiles(data_dir='data/gesture/',is_frame_based=True)
de.load_data()

## d) Generate training data

In [4]:
de.assemble_data(tolerance_range=2)

In [ ]:
de.combined_data_files_df

In [ ]:
de.display_information()

## e) Data normalization

In [5]:
X = de.X.copy()
y = de.y.copy().astype("int32")
gt = GestureTransformer(byrow=True, feature_names= list(de.feature_names))
X = gt.transform(X, verbose = True)

---------------------------------------------------------
Performing row-based transformations:
X-Mean of Hip and Shoulders:
	Row-based Mean is taken over each sample and each timestep
	Shape: (19559, 18)
Y-Mean of Hip and Shoulders:
	Row-based Mean is taken over each sample and each timestep
	Shape: (19559, 18)
Scaling based on Hip-Shoulder distance:
	Row-based distance is calculated for each sample and each timestep
	Shape: (19559, 18)
-----------------------------------------------------
X-Shift:
            min   mean  median    max   std
timestep                                   
0         196.8  295.0   282.9  511.9  37.6
1         196.8  294.9   282.8  511.9  37.6
2         196.8  294.9   282.7  511.9  37.6
3         196.8  294.9   282.7  511.9  37.6
4         196.8  294.8   282.6  511.9  37.6
5         196.8  294.8   282.6  511.9  37.6
6         196.8  294.7   282.5  511.9  37.6
7         196.8  294.7   282.5  511.9  37.6
8         196.8  294.7   282.4  511.9  37.6
9         1

# 2) Model Training

## a) Filtering 0-Labels

In [ ]:
from app_local.module import DataResampler
dr_var = DataResampler()
dr_var.fit(X,y)
dr_var.restrict_zero_labels(500, criterion="variance_lowest")
dr_var.display_information()

In [ ]:
dr = DataResampler()
dr.fit(X,y)
dr.restrict_zero_labels(500)
dr.display_information()

In [ ]:
print(dr.X.shape)
X_full_zero = np.concatenate( [dr.X, dr_var.X[(dr_var.y == 0),:,:]] )
y_full_zero = np.concatenate( [dr.y, dr_var.y[dr_var.y == 0]])
print(X_full_zero.shape, y_full_zero.shape)

In [ ]:
dr_up = DataResampler()
dr_up.fit(X_full_zero, y_full_zero)
dr_up.display_information()

In [ ]:
dr_up.upsample(n = 800, exceptions=[0,3])
dr_up.display_information()
X_final = dr_up.X
y_final = dr_up.y

## b) OneHot-Encoding of target vector

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder(sparse = False)
print(type(ohe))
target = ohe.fit_transform(y_final.reshape(-1,1))
print("Before:", y_final.shape)
print("After:", target.shape)
target[0:5,:]

## c) Train-Test-Split

## d) Model Training

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

### Train a simple SV-Classifier

In [ ]:
x_final = X_final.reshape(X_final.shape[0],-1)

model = SVC(
    random_state = 42,
    gamma = "auto"
)

model.fit(x_final, y_final)

In [ ]:
preds = model.predict(x_final)
(preds == y_final).sum()/len(y_final)

### Gridsearch with Crossvalidation

In [ ]:
model.get_params().keys()

In [ ]:
# Create the random grid vor SVM

random_grid = {
                'C': [100,10,5.0,2.0,1.0, 0.5, 0.1, 0.5, 0.01],
                'gamma': ["auto", 0.001, 0.0001],
                'kernel': ["rbf","poly"]
              }

print(random_grid)

In [ ]:
params = random_grid
scoring = "accuracy"
clf = GridSearchCV(model, params, n_jobs = -1, verbose = 2, scoring = scoring)
clf.fit(x_final, y_final)

In [ ]:
clf.cv_results_.keys()

In [ ]:
print("Index: {0}".format(clf.best_index_))
print("Score: {0}".format(clf.best_score_))
print("Params: {0}".format(clf.best_params_))

In [ ]:
from sklearn.externals import joblib
joblib.dump(clf.best_estimator_,'models/model_gesture_svm_nip120_[1000,800]_002.pkl')

In [6]:
from sklearn.externals import joblib
import os
cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)

loaded_model = joblib.load('models/model_gesture_svm_nip120_[1000,800]_001.pkl')
print(type(loaded_model))

0 C:\Users\p.schambach\Desktop\DSR\drone_steering
<class 'sklearn.model_selection._search.GridSearchCV'>


In [16]:
i = 0
pred = [-1]
while pred[0] != 1:
    pred = loaded_model.predict(X[i,:,:].reshape(1,-1))
    print(i, pred)
    i += 1


0 [0]
1 [0]
2 [0]
3 [0]
4 [0]
5 [3]
6 [3]
7 [3]
8 [3]
9 [3]
10 [3]
11 [3]
12 [3]
13 [3]
14 [3]
15 [3]
16 [3]
17 [3]
18 [3]
19 [3]
20 [3]
21 [0]
22 [0]
23 [0]
24 [0]
25 [0]
26 [0]
27 [0]
28 [0]
29 [0]
30 [0]
31 [0]
32 [0]
33 [0]
34 [0]
35 [0]
36 [0]
37 [0]
38 [0]
39 [0]
40 [0]
41 [0]
42 [0]
43 [0]
44 [0]
45 [0]
46 [0]
47 [0]
48 [0]
49 [0]
50 [0]
51 [0]
52 [0]
53 [0]
54 [0]
55 [0]
56 [0]
57 [0]
58 [0]
59 [0]
60 [0]
61 [3]
62 [3]
63 [3]
64 [3]
65 [3]
66 [3]
67 [3]
68 [3]
69 [3]
70 [3]
71 [3]
72 [3]
73 [3]
74 [3]
75 [3]
76 [3]
77 [0]
78 [0]
79 [0]
80 [0]
81 [0]
82 [0]
83 [0]
84 [0]
85 [0]
86 [0]
87 [0]
88 [0]
89 [0]
90 [0]
91 [0]
92 [0]
93 [0]
94 [0]
95 [0]
96 [0]
97 [0]
98 [0]
99 [0]
100 [0]
101 [0]
102 [0]
103 [0]
104 [0]
105 [0]
106 [0]
107 [0]
108 [0]
109 [0]
110 [3]
111 [3]
112 [3]
113 [3]
114 [3]
115 [3]
116 [3]
117 [3]
118 [3]
119 [3]
120 [3]
121 [3]
122 [3]
123 [3]
124 [3]
125 [3]
126 [0]
127 [0]
128 [0]
129 [0]
130 [0]
131 [0]
132 [0]
133 [0]
134 [0]
135 [0]
136 [0]
137 [0]
138 [0

1189 [3]
1190 [3]
1191 [3]
1192 [3]
1193 [3]
1194 [3]
1195 [3]
1196 [3]
1197 [3]
1198 [3]
1199 [3]
1200 [0]
1201 [0]
1202 [0]
1203 [0]
1204 [0]
1205 [0]
1206 [0]
1207 [0]
1208 [0]
1209 [0]
1210 [0]
1211 [0]
1212 [0]
1213 [0]
1214 [0]
1215 [0]
1216 [0]
1217 [0]
1218 [0]
1219 [3]
1220 [3]
1221 [3]
1222 [3]
1223 [3]
1224 [3]
1225 [3]
1226 [3]
1227 [3]
1228 [3]
1229 [3]
1230 [3]
1231 [3]
1232 [3]
1233 [0]
1234 [0]
1235 [0]
1236 [0]
1237 [0]
1238 [0]
1239 [0]
1240 [0]
1241 [0]
1242 [0]
1243 [0]
1244 [0]
1245 [0]
1246 [0]
1247 [0]
1248 [0]
1249 [0]
1250 [0]
1251 [3]
1252 [3]
1253 [3]
1254 [3]
1255 [3]
1256 [3]
1257 [3]
1258 [3]
1259 [3]
1260 [3]
1261 [3]
1262 [3]
1263 [3]
1264 [3]
1265 [0]
1266 [0]
1267 [0]
1268 [0]
1269 [0]
1270 [0]
1271 [0]
1272 [0]
1273 [0]
1274 [0]
1275 [0]
1276 [0]
1277 [0]
1278 [0]
1279 [0]
1280 [0]
1281 [0]
1282 [0]
1283 [3]
1284 [3]
1285 [3]
1286 [3]
1287 [3]
1288 [3]
1289 [3]
1290 [3]
1291 [3]
1292 [3]
1293 [3]
1294 [3]
1295 [3]
1296 [0]
1297 [0]
1298 [0]
1299 [0]
1

2165 [0]
2166 [0]
2167 [0]
2168 [0]
2169 [3]
2170 [3]
2171 [3]
2172 [3]
2173 [3]
2174 [3]
2175 [3]
2176 [3]
2177 [3]
2178 [3]
2179 [3]
2180 [3]
2181 [3]
2182 [0]
2183 [0]
2184 [0]
2185 [0]
2186 [0]
2187 [0]
2188 [0]
2189 [0]
2190 [3]
2191 [3]
2192 [3]
2193 [3]
2194 [3]
2195 [3]
2196 [3]
2197 [3]
2198 [3]
2199 [3]
2200 [3]
2201 [3]
2202 [0]
2203 [0]
2204 [0]
2205 [0]
2206 [0]
2207 [0]
2208 [0]
2209 [0]
2210 [0]
2211 [3]
2212 [3]
2213 [3]
2214 [3]
2215 [3]
2216 [3]
2217 [3]
2218 [3]
2219 [3]
2220 [3]
2221 [3]
2222 [3]
2223 [3]
2224 [3]
2225 [0]
2226 [0]
2227 [0]
2228 [0]
2229 [0]
2230 [0]
2231 [0]
2232 [0]
2233 [0]
2234 [3]
2235 [3]
2236 [3]
2237 [3]
2238 [3]
2239 [3]
2240 [3]
2241 [3]
2242 [3]
2243 [3]
2244 [3]
2245 [3]
2246 [3]
2247 [0]
2248 [0]
2249 [0]
2250 [0]
2251 [0]
2252 [0]
2253 [0]
2254 [0]
2255 [0]
2256 [0]
2257 [0]
2258 [0]
2259 [0]
2260 [0]
2261 [0]
2262 [0]
2263 [0]
2264 [0]
2265 [0]
2266 [0]
2267 [0]
2268 [0]
2269 [0]
2270 [0]
2271 [0]
2272 [0]
2273 [0]
2274 [3]
2275 [3]
2

3088 [0]
3089 [0]
3090 [0]
3091 [0]
3092 [0]
3093 [0]
3094 [0]
3095 [0]
3096 [0]
3097 [0]
3098 [0]
3099 [0]
3100 [0]
3101 [0]
3102 [0]
3103 [0]
3104 [0]
3105 [0]
3106 [3]
3107 [3]
3108 [3]
3109 [3]
3110 [3]
3111 [3]
3112 [3]
3113 [3]
3114 [3]
3115 [3]
3116 [3]
3117 [3]
3118 [0]
3119 [0]
3120 [0]
3121 [0]
3122 [0]
3123 [0]
3124 [0]
3125 [0]
3126 [0]
3127 [0]
3128 [0]
3129 [0]
3130 [0]
3131 [0]
3132 [0]
3133 [0]
3134 [0]
3135 [0]
3136 [3]
3137 [3]
3138 [3]
3139 [3]
3140 [3]
3141 [3]
3142 [3]
3143 [3]
3144 [3]
3145 [3]
3146 [3]
3147 [3]
3148 [3]
3149 [0]
3150 [0]
3151 [0]
3152 [0]
3153 [0]
3154 [0]
3155 [0]
3156 [0]
3157 [0]
3158 [0]
3159 [0]
3160 [0]
3161 [0]
3162 [0]
3163 [0]
3164 [0]
3165 [0]
3166 [0]
3167 [3]
3168 [3]
3169 [3]
3170 [3]
3171 [3]
3172 [3]
3173 [3]
3174 [3]
3175 [3]
3176 [3]
3177 [3]
3178 [3]
3179 [3]
3180 [0]
3181 [0]
3182 [0]
3183 [0]
3184 [0]
3185 [0]
3186 [0]
3187 [0]
3188 [0]
3189 [0]
3190 [0]
3191 [0]
3192 [0]
3193 [0]
3194 [0]
3195 [0]
3196 [0]
3197 [0]
3198 [3]
3

4037 [0]
4038 [0]
4039 [0]
4040 [0]
4041 [0]
4042 [0]
4043 [0]
4044 [6]
4045 [6]
4046 [6]
4047 [6]
4048 [6]
4049 [6]
4050 [6]
4051 [6]
4052 [6]
4053 [6]
4054 [6]
4055 [6]
4056 [6]
4057 [6]
4058 [6]
4059 [0]
4060 [0]
4061 [0]
4062 [0]
4063 [0]
4064 [0]
4065 [0]
4066 [0]
4067 [0]
4068 [0]
4069 [0]
4070 [0]
4071 [0]
4072 [0]
4073 [0]
4074 [0]
4075 [0]
4076 [0]
4077 [0]
4078 [0]
4079 [0]
4080 [0]
4081 [0]
4082 [0]
4083 [6]
4084 [6]
4085 [6]
4086 [6]
4087 [6]
4088 [6]
4089 [6]
4090 [6]
4091 [6]
4092 [6]
4093 [6]
4094 [6]
4095 [6]
4096 [6]
4097 [6]
4098 [0]
4099 [0]
4100 [0]
4101 [0]
4102 [0]
4103 [0]
4104 [0]
4105 [0]
4106 [0]
4107 [0]
4108 [0]
4109 [0]
4110 [0]
4111 [0]
4112 [0]
4113 [0]
4114 [0]
4115 [0]
4116 [0]
4117 [0]
4118 [0]
4119 [0]
4120 [0]
4121 [6]
4122 [6]
4123 [6]
4124 [6]
4125 [6]
4126 [6]
4127 [6]
4128 [6]
4129 [6]
4130 [6]
4131 [6]
4132 [6]
4133 [6]
4134 [6]
4135 [0]
4136 [0]
4137 [0]
4138 [0]
4139 [0]
4140 [0]
4141 [0]
4142 [0]
4143 [0]
4144 [0]
4145 [0]
4146 [0]
4147 [0]
4

...

9046 [0]
9047 [0]
9048 [0]
9049 [0]
9050 [0]
9051 [0]
9052 [0]
9053 [0]
9054 [0]
9055 [0]
9056 [0]
9057 [0]
9058 [0]
9059 [0]
9060 [0]
9061 [0]
9062 [0]
9063 [0]
9064 [0]
9065 [4]
9066 [4]
9067 [4]
9068 [4]
9069 [4]
9070 [4]
9071 [4]
9072 [4]
9073 [4]
9074 [4]
9075 [4]
9076 [0]
9077 [0]
9078 [0]
9079 [0]
9080 [0]
9081 [0]
9082 [0]
9083 [0]
9084 [0]
9085 [0]
9086 [0]
9087 [0]
9088 [0]
9089 [4]
9090 [4]
9091 [4]
9092 [4]
9093 [4]
9094 [4]
9095 [4]
9096 [4]
9097 [4]
9098 [4]
9099 [4]
9100 [4]
9101 [4]
9102 [4]
9103 [4]
9104 [4]
9105 [0]
9106 [0]
9107 [0]
9108 [0]
9109 [0]
9110 [0]
9111 [0]
9112 [0]
9113 [0]
9114 [0]
9115 [0]
9116 [0]
9117 [0]
9118 [0]
9119 [0]
9120 [0]
9121 [0]
9122 [0]
9123 [0]
9124 [0]
9125 [0]
9126 [0]
9127 [0]
9128 [4]
9129 [4]
9130 [4]
9131 [4]
9132 [4]
9133 [4]
9134 [4]
9135 [4]
9136 [4]
9137 [4]
9138 [4]
9139 [4]
9140 [0]
9141 [0]
9142 [0]
9143 [0]
9144 [0]
9145 [0]
9146 [0]
9147 [0]
9148 [0]
9149 [0]
9150 [0]
9151 [0]
9152 [0]
9153 [0]
9154 [0]
9155 [0]
9156 [0]
9

10031 [0]
10032 [0]
10033 [4]
10034 [4]
10035 [4]
10036 [4]
10037 [4]
10038 [4]
10039 [4]
10040 [4]
10041 [4]
10042 [4]
10043 [0]
10044 [0]
10045 [0]
10046 [0]
10047 [0]
10048 [0]
10049 [0]
10050 [0]
10051 [0]
10052 [0]
10053 [0]
10054 [0]
10055 [0]
10056 [0]
10057 [0]
10058 [0]
10059 [4]
10060 [4]
10061 [4]
10062 [4]
10063 [4]
10064 [4]
10065 [4]
10066 [4]
10067 [4]
10068 [4]
10069 [0]
10070 [0]
10071 [0]
10072 [0]
10073 [0]
10074 [0]
10075 [0]
10076 [0]
10077 [0]
10078 [0]
10079 [0]
10080 [0]
10081 [0]
10082 [0]
10083 [0]
10084 [0]
10085 [0]
10086 [0]
10087 [0]
10088 [0]
10089 [4]
10090 [4]
10091 [4]
10092 [4]
10093 [4]
10094 [4]
10095 [4]
10096 [4]
10097 [4]
10098 [4]
10099 [0]
10100 [0]
10101 [0]
10102 [0]
10103 [0]
10104 [0]
10105 [0]
10106 [0]
10107 [0]
10108 [0]
10109 [0]
10110 [0]
10111 [0]
10112 [0]
10113 [0]
10114 [0]
10115 [0]
10116 [4]
10117 [4]
10118 [4]
10119 [4]
10120 [4]
10121 [4]
10122 [4]
10123 [4]
10124 [4]
10125 [4]
10126 [0]
10127 [0]
10128 [0]
10129 [0]
10130 [0]


10923 [2]
10924 [2]
10925 [2]
10926 [2]
10927 [2]
10928 [2]
10929 [2]
10930 [2]
10931 [2]
10932 [2]
10933 [2]
10934 [2]
10935 [2]
10936 [2]
10937 [2]
10938 [2]
10939 [2]
10940 [2]
10941 [2]
10942 [2]
10943 [2]
10944 [2]
10945 [2]
10946 [2]
10947 [2]
10948 [2]
10949 [0]
10950 [0]
10951 [0]
10952 [0]
10953 [0]
10954 [2]
10955 [2]
10956 [2]
10957 [2]
10958 [2]
10959 [2]
10960 [2]
10961 [2]
10962 [2]
10963 [2]
10964 [2]
10965 [2]
10966 [2]
10967 [2]
10968 [2]
10969 [2]
10970 [2]
10971 [2]
10972 [2]
10973 [2]
10974 [2]
10975 [2]
10976 [2]
10977 [2]
10978 [2]
10979 [2]
10980 [2]
10981 [0]
10982 [2]
10983 [2]
10984 [2]
10985 [2]
10986 [2]
10987 [2]
10988 [2]
10989 [2]
10990 [2]
10991 [2]
10992 [2]
10993 [0]
10994 [0]
10995 [0]
10996 [0]
10997 [0]
10998 [0]
10999 [2]
11000 [2]
11001 [2]
11002 [2]
11003 [2]
11004 [2]
11005 [2]
11006 [2]
11007 [2]
11008 [2]
11009 [2]
11010 [2]
11011 [2]
11012 [2]
11013 [2]
11014 [2]
11015 [2]
11016 [2]
11017 [2]
11018 [2]
11019 [2]
11020 [2]
11021 [2]
11022 [2]


11894 [0]
11895 [0]
11896 [0]
11897 [0]
11898 [2]
11899 [2]
11900 [2]
11901 [2]
11902 [2]
11903 [2]
11904 [2]
11905 [2]
11906 [2]
11907 [2]
11908 [2]
11909 [2]
11910 [2]
11911 [2]
11912 [0]
11913 [0]
11914 [2]
11915 [2]
11916 [2]
11917 [2]
11918 [2]
11919 [2]
11920 [2]
11921 [2]
11922 [2]
11923 [2]
11924 [2]
11925 [2]
11926 [2]
11927 [2]
11928 [2]
11929 [2]
11930 [2]
11931 [2]
11932 [2]
11933 [2]
11934 [2]
11935 [2]
11936 [2]
11937 [2]
11938 [2]
11939 [2]
11940 [2]
11941 [2]
11942 [2]
11943 [2]
11944 [2]
11945 [2]
11946 [2]
11947 [2]
11948 [2]
11949 [2]
11950 [0]
11951 [0]
11952 [0]
11953 [0]
11954 [0]
11955 [2]
11956 [2]
11957 [2]
11958 [2]
11959 [2]
11960 [2]
11961 [2]
11962 [2]
11963 [2]
11964 [2]
11965 [2]
11966 [2]
11967 [2]
11968 [2]
11969 [0]
11970 [0]
11971 [0]
11972 [0]
11973 [0]
11974 [0]
11975 [0]
11976 [0]
11977 [2]
11978 [2]
11979 [2]
11980 [2]
11981 [2]
11982 [2]
11983 [2]
11984 [2]
11985 [2]
11986 [2]
11987 [2]
11988 [2]
11989 [2]
11990 [2]
11991 [0]
11992 [0]
11993 [0]


12784 [0]
12785 [0]
12786 [0]
12787 [0]
12788 [0]
12789 [0]
12790 [0]
12791 [0]
12792 [0]
12793 [0]
12794 [0]
12795 [0]
12796 [0]
12797 [0]
12798 [0]
12799 [0]
12800 [0]
12801 [0]
12802 [0]
12803 [0]
12804 [0]
12805 [0]
12806 [0]
12807 [0]
12808 [0]
12809 [0]
12810 [0]
12811 [0]
12812 [0]
12813 [0]
12814 [2]
12815 [2]
12816 [2]
12817 [2]
12818 [4]
12819 [4]
12820 [4]
12821 [4]
12822 [4]
12823 [0]
12824 [0]
12825 [0]
12826 [0]
12827 [0]
12828 [0]
12829 [0]
12830 [0]
12831 [0]
12832 [0]
12833 [0]
12834 [0]
12835 [0]
12836 [0]
12837 [0]
12838 [0]
12839 [0]
12840 [0]
12841 [0]
12842 [0]
12843 [0]
12844 [0]
12845 [0]
12846 [0]
12847 [0]
12848 [0]
12849 [0]
12850 [0]
12851 [0]
12852 [0]
12853 [0]
12854 [0]
12855 [0]
12856 [0]
12857 [0]
12858 [0]
12859 [0]
12860 [0]
12861 [0]
12862 [0]
12863 [0]
12864 [0]
12865 [0]
12866 [0]
12867 [0]
12868 [0]
12869 [0]
12870 [0]
12871 [0]
12872 [0]
12873 [0]
12874 [0]
12875 [0]
12876 [0]
12877 [0]
12878 [0]
12879 [0]
12880 [0]
12881 [0]
12882 [0]
12883 [0]


In [13]:
joblib.dump(x6,"models/x6.pkl")

['models/x6.pkl']

In [14]:
from sklearn.externals import joblib
joblib.dump(loaded_model.best_estimator_,'models/model_gesture_svm_nip120_[1000,800]_001.pkl')

['models/model_gesture_svm_nip120_[1000,800]_001.pkl']